In [1]:
!pip install backtesting
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916 sha256=ac91c60dc9c36b0327b20c7ce7be71268e4c6b130531efe806cfe1e281b1e904
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting
--2024-04-30 23:35:44--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2024-04-30 23:35:45--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloa

In [69]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from backtesting import Backtest, Strategy
from sklearn.mixture import GaussianMixture
import talib as ta


# รับข้อมูลประวัติของ BTC-USD
data = yf.download('BTC-USD', start='2014-09-17', end='2019-06-05')
# สร้างคอลัมน์ 'Returns' และ 'Range'
data['Returns'] = data['Close'].pct_change()
data['Range'] = (data['High'] - data['Low']) / data['Low']
data.dropna(inplace=True)

# แบ่งข้อมูลเป็นชุดการฝึกอบรมและทดสอบ
train_data = data.iloc[:int(len(data) * 0.95)]
test_data = data.iloc[int(len(data) * 0.05):]

# สร้างโมเดล Gaussian Mixture Model (GMM) ด้วย n_components = 3
gmm = GaussianMixture(n_components=3, random_state=42)
gmm.fit(train_data[['Returns', 'Range']])

# ทำการคาดการณ์ hidden states จากชุดทดสอบ
test_data.loc[:, 'hidden_state'] = gmm.predict(test_data[['Returns', 'Range']])

# เพิ่มเงินสดเริ่มต้นเป็น 100000 USD
initial_cash = 100000  # ใช้เงินสดเริ่มต้นที่เพิ่มขึ้น

# สร้างกลยุทธ์รวมที่อ้างอิงถึงผลลัพธ์ของ GMM และ ADX
class test1(Strategy):
    def init(self):
        # Calculate EMA12 using TA-Lib with correct argument name
        self.ema12 = self.I(ta.EMA, self.data.Close, timeperiod=12)
    def next(self):
        # Place a buy order when the Close price is higher than EMA12
        # if self.data.Close > self.ema12:
        if self.ema12 > self.data.Close:
            self.buy(size=0.1)

        # Place a sell order when hidden_state is 2
        elif self.data.hidden_state == 2:
            self.sell(size=0.1)


# สร้างการ backtest ด้วยกลยุทธ์รวมและข้อมูลชุดทดสอบ
bt = Backtest(test_data, test1, cash=initial_cash, commission=0.001)

# รันการ backtest และพิมพ์สถิติ
stats = bt.run()
print(stats)

# แสดงผลการ backtest
bt.plot()
plt.show()



[*********************100%%**********************]  1 of 1 completed
<ipython-input-69-ef52d0ecf301>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.loc[:, 'hidden_state'] = gmm.predict(test_data[['Returns', 'Range']])


Start                     2014-12-13 00:00:00
End                       2019-06-04 00:00:00
Duration                   1634 days 00:00:00
Exposure Time [%]                   99.204893
Equity Final [$]               2768719.101071
Equity Peak [$]                 6611690.83556
Return [%]                        2668.719101
Buy & Hold Return [%]             2118.855315
Return (Ann.) [%]                  109.882696
Volatility (Ann.) [%]              174.979882
Sharpe Ratio                         0.627973
Sortino Ratio                        2.253255
Calmar Ratio                         1.317967
Max. Drawdown [%]                  -83.372872
Avg. Drawdown [%]                  -10.783722
Max. Drawdown Duration      535 days 00:00:00
Avg. Drawdown Duration       33 days 00:00:00
# Trades                                   45
Win Rate [%]                        97.777778
Best Trade [%]                    4537.038741
Worst Trade [%]                    -17.566577
Avg. Trade [%]                    

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


In [70]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from backtesting import Backtest, Strategy
from sklearn.mixture import GaussianMixture
import talib as ta


# รับข้อมูลประวัติของ BTC-USD
data = yf.download('BTC-USD', start='2015-11-29', end='2020-06-15')


# สร้างคอลัมน์ 'Returns' และ 'Range'
data['Returns'] = data['Close'].pct_change()
data['Range'] = (data['High'] - data['Low']) / data['Low']
data.dropna(inplace=True)

# แบ่งข้อมูลเป็นชุดการฝึกอบรมและทดสอบ
train_data = data.iloc[:int(len(data) * 0.95)]
test_data = data.iloc[int(len(data) * 0.05):]

# สร้างโมเดล Gaussian Mixture Model (GMM) ด้วย n_components = 3
gmm = GaussianMixture(n_components=3, random_state=42)
gmm.fit(train_data[['Returns', 'Range']])

# ทำการคาดการณ์ hidden states จากชุดทดสอบ
test_data.loc[:, 'hidden_state'] = gmm.predict(test_data[['Returns', 'Range']])

# เพิ่มเงินสดเริ่มต้นเป็น 100000 USD
initial_cash = 100000  # ใช้เงินสดเริ่มต้นที่เพิ่มขึ้น

# สร้างกลยุทธ์รวมที่อ้างอิงถึงผลลัพธ์ของ GMM และ ADX
class test2(Strategy):
    def init(self):
        # คำนวณ ADX ด้วย TA-Lib
        self.adx = self.I(ta.ADX, self.data.High, self.data.Low, self.data.Close, timeperiod=14)

    def next(self):
        if  self.data.Close > self.adx :
            self.buy(size=0.1)

        elif self.data.hidden_state == 2 :
            self.sell(size=0.1)


# สร้างการ backtest ด้วยกลยุทธ์รวมและข้อมูลชุดทดสอบ
bt = Backtest(test_data, test2, cash=initial_cash, commission=0.001)

# รันการ backtest และพิมพ์สถิติ
stats = bt.run()
print(stats)

# แสดงผลการ backtest
bt.plot()
plt.show()


[*********************100%%**********************]  1 of 1 completed
<ipython-input-70-e3a4c6569b94>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.loc[:, 'hidden_state'] = gmm.predict(test_data[['Returns', 'Range']])


Start                     2016-02-20 00:00:00
End                       2020-06-14 00:00:00
Duration                   1576 days 00:00:00
Exposure Time [%]                   98.161065
Equity Final [$]                2172276.00504
Equity Peak [$]                4469026.244621
Return [%]                        2072.276005
Buy & Hold Return [%]             2047.200605
Return (Ann.) [%]                  103.906851
Volatility (Ann.) [%]               182.38462
Sharpe Ratio                         0.569713
Sortino Ratio                        2.004447
Calmar Ratio                          1.24705
Max. Drawdown [%]                  -83.322094
Avg. Drawdown [%]                   -8.241363
Max. Drawdown Duration      911 days 00:00:00
Avg. Drawdown Duration       33 days 00:00:00
# Trades                                   36
Win Rate [%]                            100.0
Best Trade [%]                    2207.032781
Worst Trade [%]                   2001.403959
Avg. Trade [%]                    

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


In [72]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from backtesting import Backtest, Strategy
from sklearn.mixture import GaussianMixture
import talib as ta


# รับข้อมูลประวัติของ BTC-USD
data = yf.download('BTC-USD', start='2017-09-17', end='2021-11-27')

# สร้างคอลัมน์ 'Returns' และ 'Range'
data['Returns'] = data['Close'].pct_change()
data['Range'] = (data['High'] - data['Low']) / data['Low']
data.dropna(inplace=True)

# แบ่งข้อมูลเป็นชุดการฝึกอบรมและทดสอบ
train_data = data.iloc[:int(len(data) * 0.95)]
test_data = data.iloc[int(len(data) * 0.05):]

# สร้างโมเดล Gaussian Mixture Model (GMM) ด้วย n_components = 3
gmm = GaussianMixture(n_components=3, random_state=42)
gmm.fit(train_data[['Returns', 'Range']])

# ทำการคาดการณ์ hidden states จากชุดทดสอบ
test_data.loc[:, 'hidden_state'] = gmm.predict(test_data[['Returns', 'Range']])

# เพิ่มเงินสดเริ่มต้นเป็น 100000 USD
initial_cash = 100000  # ใช้เงินสดเริ่มต้นที่เพิ่มขึ้น

# สร้างกลยุทธ์รวมที่อ้างอิงถึงผลลัพธ์ของ GMM และ ADX
class test3(Strategy):
    def init(self):
        # Calculate EMA12 using TA-Lib with correct argument name
        self.ema12 = self.I(ta.EMA, self.data.Close, timeperiod=12)
    def next(self):
        # Place a buy order when the Close price is higher than EMA12
        if self.ema12 > self.data.Close :
            self.buy(size=0.1)

        # Place a sell order when hidden_state is 2
        elif self.data.hidden_state == 2:
            self.sell(size=0.1)


# สร้างการ backtest ด้วยกลยุทธ์รวมและข้อมูลชุดทดสอบ
bt = Backtest(test_data, test3, cash=initial_cash, commission=0.001)

# รันการ backtest และพิมพ์สถิติ
stats = bt.run()
print(stats)

# แสดงผลการ backtest
bt.plot()
plt.show()


[*********************100%%**********************]  1 of 1 completed
<ipython-input-72-a7d08a290a5e>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.loc[:, 'hidden_state'] = gmm.predict(test_data[['Returns', 'Range']])


Start                     2017-12-03 00:00:00
End                       2021-11-26 00:00:00
Duration                   1454 days 00:00:00
Exposure Time [%]                    95.80756
Equity Final [$]                732671.105382
Equity Peak [$]                 841399.100823
Return [%]                         632.671105
Buy & Hold Return [%]              373.097399
Return (Ann.) [%]                   64.804974
Volatility (Ann.) [%]               97.318783
Sharpe Ratio                         0.665904
Sortino Ratio                         1.75692
Calmar Ratio                          1.25382
Max. Drawdown [%]                   -51.68602
Avg. Drawdown [%]                    -8.89071
Max. Drawdown Duration      486 days 00:00:00
Avg. Drawdown Duration       38 days 00:00:00
# Trades                                   12
Win Rate [%]                            100.0
Best Trade [%]                    1678.556625
Worst Trade [%]                    544.274536
Avg. Trade [%]                    

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


In [76]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from backtesting import Backtest, Strategy
from sklearn.mixture import GaussianMixture
import talib as ta


# รับข้อมูลประวัติของ BTC-USD
data = yf.download('BTC-USD', start='2018-04-23', end='2023-01-08')
# สร้างคอลัมน์ 'Returns' และ 'Range'
data['Returns'] = data['Close'].pct_change()
data['Range'] = (data['High'] - data['Low']) / data['Low']
data.dropna(inplace=True)

# แบ่งข้อมูลเป็นชุดการฝึกอบรมและทดสอบ
train_data = data.iloc[:int(len(data) * 0.95)]
test_data = data.iloc[int(len(data) * 0.05):]

# สร้างโมเดล Gaussian Mixture Model (GMM) ด้วย n_components = 3
gmm = GaussianMixture(n_components=3, random_state=42)
gmm.fit(train_data[['Returns', 'Range']])

# ทำการคาดการณ์ hidden states จากชุดทดสอบ
test_data.loc[:, 'hidden_state'] = gmm.predict(test_data[['Returns', 'Range']])

# เพิ่มเงินสดเริ่มต้นเป็น 100000 USD
initial_cash = 100000  # ใช้เงินสดเริ่มต้นที่เพิ่มขึ้น

# สร้างกลยุทธ์รวมที่อ้างอิงถึงผลลัพธ์ของ GMM และ ADX
class test4(Strategy):
    def init(self):
        # คำนวณ EMA12 ด้วย TA-Lib
        self.rsi = self.I(ta.RSI, self.data.Close, timeperiod=14)

    def next(self):
        if self.data.Close > self.rsi  :
            self.buy(size=0.1)

        elif self.data.hidden_state == 2 :
            self.sell(size=0.1)

# สร้างการ backtest ด้วยกลยุทธ์รวมและข้อมูลชุดทดสอบ
bt = Backtest(test_data, test4, cash=initial_cash, commission=0.001)

# รันการ backtest และพิมพ์สถิติ
stats = bt.run()
print(stats)

# แสดงผลการ backtest
bt.plot()
plt.show()


[*********************100%%**********************]  1 of 1 completed
<ipython-input-76-2c0c7588b9fc>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.loc[:, 'hidden_state'] = gmm.predict(test_data[['Returns', 'Range']])


Start                     2018-07-19 00:00:00
End                       2023-01-07 00:00:00
Duration                   1633 days 00:00:00
Exposure Time [%]                   99.020808
Equity Final [$]                235575.286358
Equity Peak [$]                 842965.808788
Return [%]                         135.575286
Buy & Hold Return [%]              127.071064
Return (Ann.) [%]                    21.09485
Volatility (Ann.) [%]               76.819957
Sharpe Ratio                         0.274601
Sortino Ratio                        0.520135
Calmar Ratio                         0.286185
Max. Drawdown [%]                  -73.710525
Avg. Drawdown [%]                    -11.2053
Max. Drawdown Duration      486 days 00:00:00
Avg. Drawdown Duration       51 days 00:00:00
# Trades                                   12
Win Rate [%]                            100.0
Best Trade [%]                     382.128014
Worst Trade [%]                    127.664114
Avg. Trade [%]                    

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


In [75]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from backtesting import Backtest, Strategy
from sklearn.mixture import GaussianMixture
import talib as ta


# รับข้อมูลประวัติของ BTC-USD
data = yf.download('BTC-USD', start='2019-07-5', end='2024-05-1')

# สร้างคอลัมน์ 'Returns' และ 'Range'
data['Returns'] = data['Close'].pct_change()
data['Range'] = (data['High'] - data['Low']) / data['Low']
data.dropna(inplace=True)

# แบ่งข้อมูลเป็นชุดการฝึกอบรมและทดสอบ
train_data = data.iloc[:int(len(data) * 0.95)]
test_data = data.iloc[int(len(data) * 0.05):]

# สร้างโมเดล Gaussian Mixture Model (GMM) ด้วย n_components = 3
gmm = GaussianMixture(n_components=3, random_state=42)
gmm.fit(train_data[['Returns', 'Range']])

# ทำการคาดการณ์ hidden states จากชุดทดสอบ
test_data.loc[:, 'hidden_state'] = gmm.predict(test_data[['Returns', 'Range']])

# เพิ่มเงินสดเริ่มต้นเป็น 100000 USD
initial_cash = 100000  # ใช้เงินสดเริ่มต้นที่เพิ่มขึ้น

# สร้างกลยุทธ์รวมที่อ้างอิงถึงผลลัพธ์ของ GMM และ ADX
class test5(Strategy):
    def init(self):
        # คำนวณ EMA12 ด้วย TA-Lib
        self.ema12 = self.I(ta.EMA, self.data.Close, timeperiod=12)

    def next(self):
        # เมื่อราคา Close สูงกว่า EMA12 ให้ออกคำสั่งซื้อ
        if self.ema12 > self.data.Close :
            self.buy(size=0.1)

        elif self.data.hidden_state == 2 :
            self.sell(size=0.1)

# สร้างการ backtest ด้วยกลยุทธ์รวมและข้อมูลชุดทดสอบ
bt = Backtest(test_data, test5, cash=initial_cash, commission=0.001)

# รันการ backtest และพิมพ์สถิติ
stats = bt.run()
print(stats)

# แสดงผลการ backtest
bt.plot()
plt.show()


[*********************100%%**********************]  1 of 1 completed
<ipython-input-75-4248db210ed1>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.loc[:, 'hidden_state'] = gmm.predict(test_data[['Returns', 'Range']])


Start                     2019-10-02 00:00:00
End                       2024-04-29 00:00:00
Duration                   1671 days 00:00:00
Exposure Time [%]                   99.222488
Equity Final [$]                551083.979642
Equity Peak [$]                 630901.073392
Return [%]                          451.08398
Buy & Hold Return [%]              660.643413
Return (Ann.) [%]                   45.147282
Volatility (Ann.) [%]               78.084181
Sharpe Ratio                         0.578187
Sortino Ratio                         1.32082
Calmar Ratio                         0.639513
Max. Drawdown [%]                  -70.596311
Avg. Drawdown [%]                   -7.687815
Max. Drawdown Duration      847 days 00:00:00
Avg. Drawdown Duration       39 days 00:00:00
# Trades                                    9
Win Rate [%]                            100.0
Best Trade [%]                      1160.2166
Worst Trade [%]                      1.917611
Avg. Trade [%]                    

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


In [44]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from backtesting import Backtest, Strategy
from sklearn.mixture import GaussianMixture
import talib as ta


# รับข้อมูลประวัติของ BTC-USD
data = yf.download('BTC-USD', start='2014-09-17', end='2024-05-1')


# สร้างคอลัมน์ 'Returns' และ 'Range'
data['Returns'] = data['Close'].pct_change()
data['Range'] = (data['High'] - data['Low']) / data['Low']
data.dropna(inplace=True)

# แบ่งข้อมูลเป็นชุดการฝึกอบรมและทดสอบ
train_data = data.iloc[:int(len(data) * 0.85)]
test_data = data.iloc[int(len(data) * 0.15):]

# สร้างโมเดล Gaussian Mixture Model (GMM) ด้วย n_components = 3
gmm = GaussianMixture(n_components=3, random_state=42)
gmm.fit(train_data[['Returns', 'Range']])

# ทำการคาดการณ์ hidden states จากชุดทดสอบ
test_data.loc[:, 'hidden_state'] = gmm.predict(test_data[['Returns', 'Range']])

# เพิ่มเงินสดเริ่มต้นเป็น 100000 USD
initial_cash = 100000  # ใช้เงินสดเริ่มต้นที่เพิ่มขึ้น

# สร้างกลยุทธ์รวมที่อ้างอิงถึงผลลัพธ์ของ GMM และ ADX
class testALL(Strategy):
    def init(self):
        # คำนวณ ADX ด้วย TA-Lib
        self.adx = self.I(ta.ADX, self.data.High, self.data.Low, self.data.Close, timeperiod=14)

    def next(self):
        if  self.data.Close > self.adx :
            self.buy(size=0.1)

        elif self.data.hidden_state == 2 :
            self.sell(size=0.1)


# สร้างการ backtest ด้วยกลยุทธ์รวมและข้อมูลชุดทดสอบ
bt = Backtest(test_data, testALL, cash=initial_cash, commission=0.001)

# รันการ backtest และพิมพ์สถิติ
stats = bt.run()
print(stats)

# แสดงผลการ backtest
bt.plot()
plt.show()


[*********************100%%**********************]  1 of 1 completed
<ipython-input-44-594092bc1c3e>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.loc[:, 'hidden_state'] = gmm.predict(test_data[['Returns', 'Range']])


Start                     2016-02-26 00:00:00
End                       2024-04-30 00:00:00
Duration                   2986 days 00:00:00
Exposure Time [%]                   99.029126
Equity Final [$]              14426058.079051
Equity Peak [$]               16593996.853028
Return [%]                       14326.058079
Buy & Hold Return [%]            13956.808743
Return (Ann.) [%]                   83.586019
Volatility (Ann.) [%]              146.761227
Sharpe Ratio                         0.569537
Sortino Ratio                        1.763354
Calmar Ratio                         1.003158
Max. Drawdown [%]                  -83.322907
Avg. Drawdown [%]                  -10.026711
Max. Drawdown Duration     1080 days 00:00:00
Avg. Drawdown Duration       47 days 00:00:00
# Trades                                   36
Win Rate [%]                            100.0
Best Trade [%]                   15258.008902
Worst Trade [%]                  13577.976366
Avg. Trade [%]                   1

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(
